In [50]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Upgrading plotly from the default 4.4.1 version to the most current 4.14.3 version is necessary, as there will otherwise be issues with the path for treemap()



In [52]:
pip install plotly==4.14.3

In [53]:
pip install pygbif

In [54]:
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
import plotly.io as pio

from pygbif import species

In [55]:
plotly.__version__

'4.14.3'

In [56]:
full_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/datasets/laji/laji_animalia.tsv',sep='\t')

In [57]:
full_data.head()

,Suositeltu yleiskielinen nimi,Tieteellinen nimi,Esiintymisen tyyppi,Uhanalaisuusluokka,Hallinnollinen asema,Synonyymit,Englanninkielinen nimi,Eliöryhmät,"Kunta, Tieteellinen nimi","Kunta, Auktorit","Pääjakso, Tieteellinen nimi","Pääjakso, Auktorit","Alajakso, Tieteellinen nimi","Alajakso, Auktorit","Luokka, Tieteellinen nimi","Luokka, Auktorit","Lahko, Tieteellinen nimi","Lahko, Auktorit","Heimo, Tieteellinen nimi","Heimo, Auktorit","Suku, Tieteellinen nimi","Suku, Auktorit","Laji, Tieteellinen nimi","Laji, Auktorit",Elinympäristöt
0,kivikkosiimahäntä,Dilta hibernica,Vakiintunut,LC (2019),NaN,NaN,NaN,Hyönteiset ja hämähäkkieläimet,Animalia,"Linnaeus, 1758",Arthropoda,"von Siebold, 1848",Hexapoda,NaN,Insecta,NaN,Archaeognatha,NaN,Machilidae,"Grassi, 1888",Dilta,"Strand, 1911",Dilta hibernica,"(Carpenter, 1907)",Ml – lehdot (myös kuusivaltaiset); Mkt – tuore...
1,rantasiimahäntä,Petrobius brevistylis,Vakiintunut,DD (2019),NaN,"Petrobius balticus Stach, 1921",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,"Linnaeus, 1758",Arthropoda,"von Siebold, 1848",Hexapoda,NaN,Insecta,NaN,Archaeognatha,NaN,Machilidae,"Grassi, 1888",Petrobius,"Leach, 1809",Petrobius brevistylis,"Carpenter, 1913","Ris – sora-, somerikko- ja kivikkorannat"
2,sokeritoukka,Lepisma saccharinum,Vakiintunut,LC (2019),NaN,"Lepisma saccharina Linnaeus, 1758",Silverpil,Hyönteiset ja hämähäkkieläimet,Animalia,"Linnaeus, 1758",Arthropoda,"von Siebold, 1848",Hexapoda,NaN,Insecta,NaN,Zygentoma,NaN,Lepismatidae,NaN,Lepisma,"Linnaeus, 1758",Lepisma saccharinum,"Linnaeus, 1758",Ir – rakennukset ja rakenteet
3,uunitoukka,Thermobia domestica,"Vakiintunut, Ihmisen vaikutuksesta",NA (2019),Kansallinen vieraslajistrategia (VN 5.3.2012),"Thermobia inquilina (Newman, 1863)",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,"Linnaeus, 1758",Arthropoda,"von Siebold, 1848",Hexapoda,NaN,Insecta,NaN,Zygentoma,NaN,Lepismatidae,NaN,Thermobia,"Bergroth, 1890",Thermobia domestica,"(Packard, 1837)",Ir – rakennukset ja rakenteet
4,paanutoukka,Ctenolepisma lineatum,Ihmisen vaikutuksesta,NA (2019),NaN,"Ctenolepisma lineata (Fabricius, 1775)",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,"Linnaeus, 1758",Arthropoda,"von Siebold, 1848",Hexapoda,NaN,Insecta,NaN,Zygentoma,NaN,Lepismatidae,NaN,Ctenolepisma,"Escherich, 1905",Ctenolepisma lineatum,"Fabricius, 1775",Ir – rakennukset ja rakenteet


In [58]:
df = full_data.drop(['Kunta, Auktorit', 'Pääjakso, Auktorit', 'Alajakso, Auktorit', 'Luokka, Auktorit', 'Lahko, Auktorit', 'Heimo, Auktorit', 'Suku, Auktorit', 'Laji, Auktorit'], axis = 1)

In [59]:
df = df.rename(columns = {'Kunta, Tieteellinen nimi': 'Kunta', 'Pääjakso, Tieteellinen nimi': 'Pääjakso', 'Alajakso, Tieteellinen nimi': 'Alajakso',
                          'Luokka, Tieteellinen nimi': 'Luokka', 'Lahko, Tieteellinen nimi': 'Lahko', 'Heimo, Tieteellinen nimi': 'Heimo',
                          'Suku, Tieteellinen nimi': 'Suku', 'Laji, Tieteellinen nimi': 'Laji', 'Uhanalaisuusluokka': 'ConservationStatus',
                          'Suositeltu yleiskielinen nimi': 'Name_FIN', 'Englanninkielinen nimi': 'Name_ENG', 'Hallinnollinen asema': 'HallinnollinenAsema'}, inplace = False)
df.head()

,Name_FIN,Tieteellinen nimi,Esiintymisen tyyppi,ConservationStatus,HallinnollinenAsema,Synonyymit,Name_ENG,Eliöryhmät,Kunta,Pääjakso,Alajakso,Luokka,Lahko,Heimo,Suku,Laji,Elinympäristöt
0,kivikkosiimahäntä,Dilta hibernica,Vakiintunut,LC (2019),NaN,NaN,NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Archaeognatha,Machilidae,Dilta,Dilta hibernica,Ml – lehdot (myös kuusivaltaiset); Mkt – tuore...
1,rantasiimahäntä,Petrobius brevistylis,Vakiintunut,DD (2019),NaN,"Petrobius balticus Stach, 1921",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Archaeognatha,Machilidae,Petrobius,Petrobius brevistylis,"Ris – sora-, somerikko- ja kivikkorannat"
2,sokeritoukka,Lepisma saccharinum,Vakiintunut,LC (2019),NaN,"Lepisma saccharina Linnaeus, 1758",Silverpil,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Zygentoma,Lepismatidae,Lepisma,Lepisma saccharinum,Ir – rakennukset ja rakenteet
3,uunitoukka,Thermobia domestica,"Vakiintunut, Ihmisen vaikutuksesta",NA (2019),Kansallinen vieraslajistrategia (VN 5.3.2012),"Thermobia inquilina (Newman, 1863)",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Zygentoma,Lepismatidae,Thermobia,Thermobia domestica,Ir – rakennukset ja rakenteet
4,paanutoukka,Ctenolepisma lineatum,Ihmisen vaikutuksesta,NA (2019),NaN,"Ctenolepisma lineata (Fabricius, 1775)",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Zygentoma,Lepismatidae,Ctenolepisma,Ctenolepisma lineatum,Ir – rakennukset ja rakenteet


In [60]:
#testing mislabeled specie in fig, works here.
df[df['Name_FIN'] == 'alppikiitäjä']

,Name_FIN,Tieteellinen nimi,Esiintymisen tyyppi,ConservationStatus,HallinnollinenAsema,Synonyymit,Name_ENG,Eliöryhmät,Kunta,Pääjakso,Alajakso,Luokka,Lahko,Heimo,Suku,Laji,Elinympäristöt
26814,alppikiitäjä,Tachymarptis melba,NaN,NaN,NaN,Apus melba,Alpine Swift,Linnut,Animalia,Chordata,Vertebrata,Aves,Caprimulgiformes,Apodidae,Tachymarptis,Tachymarptis melba,NaN


In [61]:
df.iloc[14050]

Name_FIN                                                  tuonensirvikäs
Tieteellinen nimi                                    Oligotricha striata
Esiintymisen tyyppi                                 Vakiintunut, Yleinen
ConservationStatus                                             LC (2019)
HallinnollinenAsema                                                  NaN
Synonyymit                                                           NaN
Name_ENG                                                             NaN
Eliöryhmät             Hyönteiset ja hämähäkkieläimet, Vesihyönteiset...
Kunta                                                           Animalia
Pääjakso                                                      Arthropoda
Alajakso                                                        Hexapoda
Luokka                                                           Insecta
Lahko                                                        Trichoptera
Heimo                                              

In [63]:
#df = df.replace(np.nan, None, regex=True)
# testing whether removes empty leaves will resolve mismatched labels in fig
df.dropna(subset = ['Kunta', 'Pääjakso', 'Alajakso', 'Luokka', 'Lahko', 'Heimo', 'Suku', 'Laji'])

,Name_FIN,Tieteellinen nimi,Esiintymisen tyyppi,ConservationStatus,HallinnollinenAsema,Synonyymit,Name_ENG,Eliöryhmät,Kunta,Pääjakso,Alajakso,Luokka,Lahko,Heimo,Suku,Laji,Elinympäristöt,extinctness
0,kivikkosiimahäntä,Dilta hibernica,Vakiintunut,LC (2019),NaN,NaN,NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Archaeognatha,Machilidae,Dilta,Dilta hibernica,Ml – lehdot (myös kuusivaltaiset); Mkt – tuore...,1
1,rantasiimahäntä,Petrobius brevistylis,Vakiintunut,DD (2019),NaN,"Petrobius balticus Stach, 1921",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Archaeognatha,Machilidae,Petrobius,Petrobius brevistylis,"Ris – sora-, somerikko- ja kivikkorannat",1
2,sokeritoukka,Lepisma saccharinum,Vakiintunut,LC (2019),NaN,"Lepisma saccharina Linnaeus, 1758",Silverpil,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Zygentoma,Lepismatidae,Lepisma,Lepisma saccharinum,Ir – rakennukset ja rakenteet,1
3,uunitoukka,Thermobia domestica,"Vakiintunut, Ihmisen vaikutuksesta",NA (2019),Kansallinen vieraslajistrategia (VN 5.3.2012),"Thermobia inquilina (Newman, 1863)",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Zygentoma,Lepismatidae,Thermobia,Thermobia domestica,Ir – rakennukset ja rakenteet,1
4,paanutoukka,Ctenolepisma lineatum,Ihmisen vaikutuksesta,NA (2019),NaN,"Ctenolepisma lineata (Fabricius, 1775)",NaN,Hyönteiset ja hämähäkkieläimet,Animalia,Arthropoda,Hexapoda,Insecta,Zygentoma,Lepismatidae,Ctenolepisma,Ctenolepisma lineatum,Ir – rakennukset ja rakenteet,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27434,särmäneula,Syngnathus typhle,Vakiintunut,LC (2019),NaN,NaN,Broadnosed pipefish,Kalat,Animalia,Chordata,Vertebrata,Actinopterygii,Syngnathiformes,Syngnathidae,Syngnathus,Syngnathus typhle,Vi – Itämeri,1
27435,merinahkiainen,Petromyzon marinus,Säännöllinen harhailija,NaN,NaN,NaN,Sea lamprey,Kalat,Animalia,Chordata,Vertebrata,Cephalaspidomorphi,Petromyzontiformes,Petromyzontidae,Petromyzon,Petromyzon marinus,NaN,1
27436,nahkiainen,Lampetra fluviatilis,Vakiintunut,NT (2019),"EU:n luontodirektiivin II-liite, Suomi saanut ...",NaN,River lamprey,Kalat,Animalia,Chordata,Vertebrata,Cephalaspidomorphi,Petromyzontiformes,Petromyzontidae,Lampetra,Lampetra fluviatilis,Vj – joet; Vi – Itämeri; Vk – kosket; Vsk – ka...,1
27437,pikkunahkiainen,Lampetra planeri,Vakiintunut,LC (2019),"EU:n luontodirektiivin II-liite, EU:n luontodi...",NaN,Brook lamprey,Kalat,Animalia,Chordata,Vertebrata,Cephalaspidomorphi,Petromyzontiformes,Petromyzontidae,Lampetra,Lampetra planeri,Vp – purot ja norot; Vj – joet,1


In [64]:
df.ConservationStatus.unique()

array(['LC (2019)', 'DD (2019)', 'NA (2019)', nan, 'EN (2019)',
       'VU (2019)', 'NT (2019)', 'CR (2019)', 'NE (2019)', 'RE (2019)',
       'NA (2010)', 'NE (2010)', 'LC (2010)', 'DD (2010)', 'EN (2000)',
       'NT (2000)', 'LC (2000)', 'VU (2010)', 'NT (2010)', 'VU (2015)'],
      dtype=object)

In [65]:
# Endangernment Scores
#LC (Least Concern)            100
#NT (Near threatened)          66
#VU (Vulnerable)               50
#EN (Endangered)               33
#CR (Critically endangered)    15
#EW (Extinct in the wild)      5
#EX (Extinct)                  1

#DD (Data deficient)           50
#NE (Near Extinct)             10

#RE (Regionally Extinct)       1
#NA (Not Applicable)           50
df_conserv = df.copy()
df_conserv["ConservationStatus"] = df["ConservationStatus"].map(lambda x: "100" if "LC" in  str(x) else "66" if "NT" in str(x) else "50" if "VU" in str(x) 
                                                                else "33" if "EN" in str(x) else "15" if "CR" in str(x) else "50" if "DD" in str(x) 
                                                                else "10" if "NE" in str(x) else "1" if "RE" in str(x) else "50" if "NA" in str(x) else "50") 

df_conserv["ConservationStatusText"] = df["ConservationStatus"].map(lambda x: "Least Concern" if "LC" in str(x) else "Near Threatened" if "NT" in str(x) 
                                                                    else "Vulnerable" if "VU" in str(x) else "Endangered" if "EN" in str(x) 
                                                                    else "Critically Endangered" if "CR" in str(x) else "Data Deficient" if "DD" in str(x) 
                                                                    else "Near Extinct" if "NE" in str(x) else "Regionally Extinct" if "RE" in str(x) 
                                                                    else "Not Applicable" if "NA" in str(x) else "")

In [66]:
df_conserv[['ConservationStatusText', 'ConservationStatus']].head(), 
#df_conserv['ConservationStatusText'].head()

(  ConservationStatusText ConservationStatus
 0          Least Concern                100
 1         Data Deficient                 50
 2          Least Concern                100
 3         Not Applicable                 50
 4         Not Applicable                 50,)

In [67]:
# using the gbif api search to get most likely species id
def get_gbif(x):
  name = species.name_suggest(q=x)
  if len(name)!=0:
   return name[0].get('speciesKey')
  return ''

In [68]:
import requests
# using the gbif api to get species ids that have similar name
def get_gbif_api(x):
  url = "https://api.gbif.org/v1/species/match?verbose=false&scientificName=" + x
  id = requests.get(url).json().get('speciesKey')
  return id

In [69]:
get_gbif("Hyla arborea")

2427573

In [70]:
# unfortunately using the api was too slow for the number of species data involved.
#df_vertebrata['gbif_link'] = df_vertebrata['Laji'].apply(lambda x: get_gbif(x))

In [71]:
#Adding text data to the treemap boxes
df_vertebrata = df_conserv[df['Alajakso'].isin(['Vertebrata'])]
df_vertebrata.HallinnollinenAsema = df_vertebrata.HallinnollinenAsema.apply(lambda x: 'introduced species/vieraslaji' if 'vieras' in str(x) else '')


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [72]:
# test get_gbif_api function
pd.concat([df_vertebrata['Laji'].head(10), df_vertebrata['Laji'].head(10).apply(lambda x: get_gbif_api(x))], axis=1)

,Laji,Laji
26719,Bombina variegata,2423435
26720,Bufo bufo,5217160
26721,Pseudepidalea viridis,9529916
26722,Hyla arborea,2427573
26723,Rana arvalis,2426789
26724,Rana temporaria,2426805
26725,Pelophylax esculentus,2426654
26726,Pelophylax lessonae,2426654
26727,Pelophylax ridibundus,2426661
26728,Triturus cristatus,2431885


In [73]:
#test
df_vertebrata[df_vertebrata['Name_FIN'].map(str) == 'alppikiitäjä']

,Name_FIN,Tieteellinen nimi,Esiintymisen tyyppi,ConservationStatus,HallinnollinenAsema,Synonyymit,Name_ENG,Eliöryhmät,Kunta,Pääjakso,Alajakso,Luokka,Lahko,Heimo,Suku,Laji,Elinympäristöt,extinctness,ConservationStatusText
26814,alppikiitäjä,Tachymarptis melba,NaN,50,,Apus melba,Alpine Swift,Linnut,Animalia,Chordata,Vertebrata,Aves,Caprimulgiformes,Apodidae,Tachymarptis,Tachymarptis melba,NaN,1,


In [74]:
# generate text for treemap boxes
df_vertebrata['text'] = "FIN: " + df_vertebrata['Name_FIN'].map(str) + "<br>" + "ENG: " + df_vertebrata['Name_ENG'].map(str) + "<br>" + df_vertebrata['ConservationStatusText'].map(str) + "<br>" + "<a href='https://www.gbif.org/species/search?q=" + df_vertebrata['Laji'].map(str) + "'>gbif link</a><br>" + (df_vertebrata['HallinnollinenAsema'].map(str))
df_vertebrata.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Name_FIN,Tieteellinen nimi,Esiintymisen tyyppi,ConservationStatus,HallinnollinenAsema,Synonyymit,Name_ENG,Eliöryhmät,Kunta,Pääjakso,Alajakso,Luokka,Lahko,Heimo,Suku,Laji,Elinympäristöt,extinctness,ConservationStatusText,text
26719,vuoristokellosammakko,Bombina variegata,Ihmisen vaikutuksesta,50,introduced species/vieraslaji,NaN,Yellow-bellied toad,"Matelijat ja sammakkoeläimet, Sammakkoeläimet",Animalia,Chordata,Vertebrata,Amphibia,Anura,Bombinatoridae,Bombina,Bombina variegata,NaN,1,Not Applicable,FIN: vuoristokellosammakko<br>ENG: Yellow-bell...
26720,rupikonna,Bufo bufo,Vakiintunut,100,,NaN,Common toad,"Matelijat ja sammakkoeläimet, Sammakkoeläimet",Animalia,Chordata,Vertebrata,Amphibia,Anura,Bufonidae,Bufo,Bufo bufo,M – Metsät; S – Suot; R – Rannat; Vs – järvet ...,1,Least Concern,FIN: rupikonna<br>ENG: Common toad<br>Least Co...
26721,viherkonna,Pseudepidalea viridis,Satunnainen harhailija,50,,"Bufo viridis Laurenti, 1768",European Green Toad,"Matelijat ja sammakkoeläimet, Sammakkoeläimet",Animalia,Chordata,Vertebrata,Amphibia,Anura,Bufonidae,Pseudepidalea,Pseudepidalea viridis,NaN,1,Not Applicable,FIN: viherkonna<br>ENG: European Green Toad<br...
26722,euroopanlehtisammakko,Hyla arborea,Ihmisen vaikutuksesta,50,introduced species/vieraslaji,NaN,European tree frog,"Matelijat ja sammakkoeläimet, Sammakkoeläimet",Animalia,Chordata,Vertebrata,Amphibia,Anura,Hylidae,Hyla,Hyla arborea,NaN,1,,FIN: euroopanlehtisammakko<br>ENG: European tr...
26723,viitasammakko,Rana arvalis,Vakiintunut,100,,NaN,Moor frog,"Matelijat ja sammakkoeläimet, Sammakkoeläimet",Animalia,Chordata,Vertebrata,Amphibia,Anura,Ranidae,Rana,Rana arvalis,M – Metsät; S – Suot; R – Rannat; I – Perinney...,1,Least Concern,FIN: viitasammakko<br>ENG: Moor frog<br>Least ...


In [75]:
#test
df_vertebrata[df_vertebrata['Name_FIN'] == 'pikkuvarpunen']

,Name_FIN,Tieteellinen nimi,Esiintymisen tyyppi,ConservationStatus,HallinnollinenAsema,Synonyymit,Name_ENG,Eliöryhmät,Kunta,Pääjakso,Alajakso,Luokka,Lahko,Heimo,Suku,Laji,Elinympäristöt,extinctness,ConservationStatusText,text
27071,pikkuvarpunen,Passer montanus,Vakiintunut,100,,NaN,Eurasian Tree Sparrow,Linnut,Animalia,Chordata,Vertebrata,Aves,Passeriformes,Passeridae,Passer,Passer montanus,"Ir – rakennukset ja rakenteet; Ip - puistot, p...",1,Least Concern,FIN: pikkuvarpunen<br>ENG: Eurasian Tree Sparr...


In [77]:
fig = px.treemap(df_vertebrata, path=['Kunta', 'Pääjakso', 'Alajakso', 'Luokka', 'Lahko',	'Heimo', 'Suku', 'Laji'], values='ConservationStatus',
                  color='ConservationStatus', hover_data=['Name_FIN'], 
                  color_continuous_scale='Hot',  color_continuous_midpoint=50
                  #,color_continuous_midpoint=np.average(df['ConservationStatus'], weights=df['fake_val'])
                  )
#fig.data[0].hoverinfo = 'label+value+text'
fig.data[0].hovertemplate = '%{label}<br>Total Conservation Score: %{value}<br>Average Conservation: %{color}%'
#fig.show()
fig.update_traces(maxdepth=3, 
                  text = df_vertebrata['text'],
 #                 text='Suositeltu yleiskielinen nimi', textinfo=text, 
                  selector=dict(type='treemap'))
fig.update_layout(title_text="Endangered Animals in Finland according to the <a href='https://laji.fi/'>Finnish Biodiversity Information Facility</a>")
fig.show()



ValueError: ignored

In [ ]:
# examination of fig contents
#fig.data[0]
#fig.data[0].labels.shape,
fig.data[0].labels

In [ ]:
species.name_suggest(q='puma concolor')[0]

In [ ]:
#It would be easy to generate links to species profiles on gbif with this if it wasn't so slow.
"https://www.gbif.org/species/" + str(species.name_suggest(q='puma concolor')[0].get('speciesKey'))

In [ ]:
dir(species.name_suggest(q='puma concolor')[0])

In [ ]:
pio.write_html(fig, file='index.html', auto_open=True)